<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [4]:
import os
os.getcwd()
KDD2020-Debiasing-Team666/data/underexpose_train/underexpose_item_feat.csv

'/Users/ness001/OneDrive/推荐系统/KDD2020-Debiasing-Team666/code/ness001'

In [8]:
import sys

print(sys.version, sys.platform, sys.executable)
from tqdm import tqdm

import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
pd.set_option('display.precision',16)

train_path = '../../data/underexpose_train/'
test_path = '../../data/'


col_name = ['item_id']
for i in range(0, 128):
    col_name.append('tv' + str(i))
for i in range(0, 128):
    col_name.append('iv' + str(i))

itemft = pd.read_csv(train_path + 'underexpose_item_feat.csv', low_memory=False, names=col_name)
itemft = itemft.replace('[\[\]]', '', regex=True)  # regex=True is the key



3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)] darwin /Users/ness001/anaconda3/envs/newborn/bin/python


In [ ]:
# dft = pd.DataFrame()
# for p in tqdm(phase):
#     # 获取test用户的近n次点击
#     temp_ = all_click_df[p][0][all_click_df[p][0].user_id.isin(all_click_df[p][1].user_id)].groupby('user_id')['item_id'].agg(list)
#     temp_ = pd.DataFrame(temp_)
#     for i in range(0, 5):
#         temp_[f'last_{i+1}'] = temp_.item_id.str.get(-(i+1))
#     dft = dft.append(temp_.reset_index())
#     # 计算每个phase的item_id cnt
#     t_ = all_click_df[p][0].groupby('item_id')['user_id'].count()
#     df.loc[df.phase==p, 'item_cnt'] = df.item_id_pred.map(lambda x: t_[x] if x in t_ else 0)

In [ ]:
vec = itemft.iloc[:, 1:].to_numpy()
vec.shape

In [ ]:
form 

In [4]:


chunk_size = 500
matrix_len = vec.shape[0]


def sim_chunk(start, end):
    if end > matrix_len:
        end = matrix_len
    return cosine_similarity(vec[start:end], vec)


import numpy as np

sim_matrix = np.ndarray(shape=(matrix_len, matrix_len))

# ---------------------------------------------------------------------

for start in tqdm(range(0, matrix_len, chunk_size)):
    cos_sim_chunk = sim_chunk(start, start + chunk_size)
    if (start + chunk_size) <= matrix_len:
        sim_matrix[start:start + chunk_size] = cos_sim_chunk
    else:
        sim_matrix[start:matrix_len] = cos_sim_chunk


100%|██████████| 218/218 [03:58<00:00,  1.10s/it]


In [16]:
results = {}

recall_num=500
for idx, row in tqdm(itemft[['item_id']].iterrows(),total=sim_matrix.shape[0]):
    similar_indices = sim_matrix[idx].argsort()[
                      :-recall_num:-1]  # ascending return index, : means reverse order, -100:-1 means last 100-1 to last 1-1 position
    similar_items = [(sim_matrix[idx][i], itemft['item_id'][i]) for i in similar_indices]

    results[row['item_id']] = similar_items[1:]
    
with open('result_vec.txt','w') as f:
    print(results,file=f)

100%|██████████| 108916/108916 [30:24<00:00, 59.70it/s] 


In [17]:
len(similar_items[1:])

498

In [18]:
# simple test
def recommend(item_id, top_num):
    print("Recommending " + str(top_num) + " products similar to " + str(item_id) + "...")
    print("-------")
    recs = results[item_id][:top_num]
    return recs


print(recommend(item_id=1, top_num=5))

# --- ------ ----------------------

p = 6
user_last = pd.read_csv(test_path + 'underexpose_test_click-{}.csv'.format(p),names=['user_id','item_id','time'])
user_last=user_last.sort_values(['user_id', 'time'],ascending=[True,False]).reset_index(drop=True)
user_true = user_last.drop_duplicates(['user_id'], keep='first')
user_train = user_last.iloc[user_true.index+1,:]

Recommending 5 products similar to 1...
-------
[(0.7412481239635497, 60268), (0.7154533810786937, 81448), (0.6957638213171138, 75224), (0.6946299147484594, 24871), (0.6573989783528436, 26004)]


In [1]:
# user_path='../../user_data/'
# results= open(user_path+'result_vec.txt').read()

In [19]:
recall_num=450
itemlist=list(itemft.item_id.unique())
recall_frame = pd.DataFrame(columns=['user_id', 'item_id_pred', 'score', 'rank', 'item_id_true'],index=range(0,recall_num*len(user_train.user_id.unique())))
row_num=0
for user in tqdm(user_train.user_id.unique()):
    latest_item = user_train.loc[user_train.user_id == user, 'item_id'].values[0]
    true_item = user_true.loc[user_true.user_id == user, 'item_id'].values[0]
    if latest_item in itemlist: 
        recs = results[latest_item][:recall_num]
        for i in range(0, recall_num):
            
            recall_frame.iloc[row_num]['user_id'] = user
            recall_frame.iloc[row_num]['item_id_pred'] = recs[i][1]
            recall_frame.iloc[row_num]['score'] = recs[i][0]
            recall_frame.iloc[row_num]['rank'] = i+1
            recall_frame.iloc[row_num]['item_id_true'] = true_item
            row_num+=1
    else:
#         print(user,latest_item,latest_item in itemlist)
        recall_frame.iloc[row_num]['user_id'] = user
        recall_frame.iloc[row_num]['item_id_pred'] = np.nan
        recall_frame.iloc[row_num]['score'] = np.nan
        recall_frame.iloc[row_num]['rank'] = np.nan
        recall_frame.iloc[row_num]['item_id_true'] = true_item
        row_num+=1
recall_frame

recall_frame=recall_frame.dropna(how='all').astype('float32')
sum(recall_frame.item_id_pred == recall_frame.item_id_true)/1821

100%|██████████| 1821/1821 [07:17<00:00,  4.17it/s]


0.13948380010982978

In [ ]:
recall_frame.to_csv('recall_allvec_df.csv',index=False)

In [ ]:
# import faiss

# vec=np.ascontiguousarray(vec.astype('float32'))

# dim=vec.shape[1]
# k=500 # recall 
# nlist=100
# quantizer = faiss.IndexFlatL2(dim)
# idx=faiss.IndexIVFPQ(quantizer,dim,nlist,10,8)
# idx.nprobe =10

# idx.train(vec)
# idx.add(vec)
# d,i=idx.search(vec,100)